<a href="https://colab.research.google.com/github/Noptus/JewelClassification/blob/main/PandoraClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import requests
import json
import base64

In [ ]:
pandora_folder_path = '/content/drive/My Drive/Personnel/Pandora'  # Update with your folder's actual location
os.chdir(pandora_folder_path)

In [ ]:
!pip install ai_api_client_sdk gradio==4.14.0 cfenv openai huggingface_hub generative-ai-hub-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found exist

In [ ]:
def encode_image_from_url(image_url):
    """Download and encode image to base64 format from URL."""
    response = requests.get(image_url)
    if response.status_code == 200:
        return base64.b64encode(response.content).decode("utf-8")
    else:
        raise Exception(f"Failed to download image. Status code: {response.status_code}")

In [ ]:
def encode_image_from_file(image_path):
    """Encode image to base64 format from local file path."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


In [ ]:
def get_oauth_token():
    auth_url = os.environ["AICORE_AUTH_URL"]
    client_id = os.environ["AICORE_CLIENT_ID"]
    client_secret = os.environ["AICORE_CLIENT_SECRET"]
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(auth_url, data=data)
    response.raise_for_status()
    token_data = response.json()
    return token_data["access_token"]
token = get_oauth_token()

In [ ]:
def get_file_id_from_path(file_path):
    """Gets the file ID for a file in Google Drive given its path."""
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            auth.authenticate_user()
            creds = None  # Modified to remove previous credentials
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    # Split the file path into parts
    parts = file_path.split('/')

    # Start from the 'My Drive' folder
    folder_id = 'root'

    # Iterate through the path parts to find the file
    for part in parts[4:]:  # Starting from 'Personnel'
        query = f"name='{part}' and '{folder_id}' in parents and trashed=false"
        results = service.files().list(q=query, fields="files(id, mimeType)").execute()
        files = results.get('files', [])

        if files:
            file = files[0]
            if file['mimeType'] == 'application/vnd.google-apps.folder':  # If it's a folder
                folder_id = file['id']  # Update folder_id for the next iteration
            else:
                return file['id']  # If it's the file, return its ID
        else:
            return None  # File or folder not found

    return None  # File not found


In [ ]:
file_path = '/content/drive/MyDrive/Personnel/Pandora/Data_Intern/R9GM00016/Good/Good 1.JPG'
file_id = get_file_id_from_path(file_path)
file_id

'1Y8i1BldzUm7Fuj1pr4uYUXJXtg3glfhu'

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
import os
import pickle

def get_public_url(file_id):
    """Gets the public URL for a file in Google Drive."""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            auth.authenticate_user()
            creds = None  # Modified to remove previous credentials
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    # Get the file metadata with webContentLink
    file = service.files().get(fileId=file_id, fields='webContentLink').execute()

    public_url = file.get('webContentLink')
    return public_url

# Example Usage:
file_id = '1Y8i1BldzUm7Fuj1pr4uYUXJXtg3glfhu'  # Replace with your file ID
public_url = get_public_url(file_id)
print(f"Public URL: {public_url}")

Public URL: https://drive.google.com/uc?id=1Y8i1BldzUm7Fuj1pr4uYUXJXtg3glfhu&export=download


In [ ]:
def call_ai_core_api(prompt_text, img1,img2,img3,img4):
    deployment_url = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d49994698792fa4c"  # Replace with your deployment URL
    resource_group = "default"
    token = get_oauth_token()
    #image_base64 = encode_image_from_url(image_url)
    headers = {
        'AI-Resource-Group': "default",
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {token}'
    }

    data = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt_text
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": img1
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": img2
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": img3
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": img4
                        }
                    }
                ]
            }
        ],
        "max_tokens": 100
    }

    response = requests.post(f"{deployment_url}/chat/completions?api-version=2023-05-15",
                             headers=headers, json=data)
    response.raise_for_status()
    answer = response.json()['choices'][0]['message']['content']
    print(answer)

    return answer

In [ ]:
prompt_text = "describe the scene"
image_url = "https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/Good/Good%2010.JPG"
call_ai_core_api(prompt_text, image_url)

In [ ]:
prompt_text = "Analyze the provided images of silver beads and identify the visual differences between the 'good' and 'bad' beads. Describe the specific defects or issues you observe, such as scratches, foreign material, or unclear stamps. Provide a detailed comparison of the characteristics that distinguish the high-quality bead from the others. Your response should focus on objective, observable details rather than making subjective judgments."

prompt_text = "Analyze the following images of silver beads and for each one: Provide a confidence score between 1 and 5 (5 being highest confidence) in your assessment.Identify if there are any defects present, such as scratches, foreign material, or unclear stamps. Describe the quality of the pearl, and if there are defects, the type of defect, if any. Output your analysis for each image in a JSON format like this:{'quality': [1-5],'confidence': [1-5],'defect': '[defect type]'}. If there are no defects, don't provide an analysis."

beginning = "Analyze the following images of silver beads and for each one: Provide a quality score between 1 and 10 (10 being highest quality).Identify if there are any defects present, such as:"
defects = "not a perfect heart shape, scun stain, gem is not shiny enough. The shape of the heart needs to be exact, symetrical and nice, especially at the bottom. It needs to be shining bright; dark areas indicate it isn't."
finish = "Just provide this number and a very short comment on the quality/deffect for the FINAL IMAGE, in a json format ('quality_of_last_image':1,'comment':'last image has a scratch'). OUTPUT NOTHING MORE THAN THIS JSON. I have given you examples of a good image, 1 wrong image with a broken crack, 1 wrong image that is not shiny enough, and the last image is the one you have to classify."

prompt_text = beginning + defects + finish


In [ ]:
img1 = "https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/Good/Good%2010.JPG"
img2 = "https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/NotGood/Broken_Crack%204.JPG"
img3 = "https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/NotGood/Gem%20not%20shing%203.JPG"


In [ ]:
def process_good_images(repo_dir):
    for i in range(1, 29):  # Iterate from 1 to 29
        image_filename = f'Good%20{i}.JPG'
        image_path = os.path.join(repo_dir, image_filename)
        print(image_path)
        call_ai_core_api(prompt_text, img1,img2,img3,image_path)
        call_ai_core_api(prompt_text, img1,img2,img3,image_path)
        call_ai_core_api(prompt_text, img1,img2,img3,image_path)
        print()

In [ ]:
def process_image_groups(repo_dir):
    image_groups = {
        "Gem%20not%20shing": range(2, 6),
        "Scun_Stain": range(1, 6),
        "Shape%20not%20good": range(1, 11)
    }

    for group_name, image_range in image_groups.items():
        for i in image_range:
            try:
              image_filename = f"{group_name}%20{i}.JPG"
              image_path = os.path.join(repo_dir, image_filename)
              print(image_path)
              call_ai_core_api(prompt_text, img1,img2,img3,image_path)
              call_ai_core_api(prompt_text, img1,img2,img3,image_path)
              call_ai_core_api(prompt_text, img1,img2,img3,image_path)

              print()
              print()
              print()
            except Exception as e:
              print("url not found: "+image_path)

In [ ]:
print("Good images")
repo_dir = 'https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/Good/'  # The directory where you downloaded the repo
process_good_images(repo_dir)

Good images
https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/Good/Good%201.JPG
```json
{'quality_of_last_image': 7, 'comment': 'last image has a scratch'}
```
```json
{'quality_of_last_image': 7, 'comment': 'last image has a scratch'}
```
```json
{
  'quality_of_last_image': 1,
  'comment': 'last image has a scratch'
}
```

https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/Good/Good%202.JPG
```json
{"quality_of_last_image":9,"comment":"last image has a scratch"}
```
```json
{'quality_of_last_image': 9, 'comment': 'no defects'}
```
```json
{'quality_of_last_image':7,'comment':'last image has a scratch'}
```

https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/Good/Good%203.JPG
```json
{'quality_of_last_image': 9, 'comment': 'last image has no major defects'}
```
```json
{'quality_of_last_image': 1, 'comment': 'last image has a scratch'}
```
```json
{
  'quality_of_last_image': 1,
  'comment': 'last image not a 

In [ ]:
print("Bad images")
repo_dir = 'https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/NotGood/'  # The directory where you downloaded the repo
process_image_groups(repo_dir)

Bad images
https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/NotGood/Gem%20not%20shing%202.JPG
```json
{'quality_of_last_image': 1, 'comment': 'last image has a scratch'}
```
```json
{'quality_of_last_image': 1, 'comment': 'last image has a scratch'}
```
```json
{'quality_of_last_image':1,'comment':'last image has a scratch'}
```



https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/NotGood/Gem%20not%20shing%203.JPG
```json
{'quality_of_last_image':10,'comment':'none'}
```
```json
{'quality_of_last_image': 7, 'comment': 'last image shape is not perfect'}
```
```json
{'quality_of_last_image': 1, 'comment': 'last image has a scratch'}
```



https://raw.githubusercontent.com/raphaelsap/jewelClassifier/main/R9GM00016/NotGood/Gem%20not%20shing%204.JPG
```json
{'quality_of_last_image': 7, 'comment': 'last image has a scratch'}
```
```json
{'quality_of_last_image': 8, 'comment': 'last image has a slight lack of symmetry'}
```
```json
{'quali